# Email

与网页通过HTTP协议传输一样，邮件是通过SMTP（simple mail transfer protocol）简单邮件传输协议传输的。

虽然用python发邮件很容易， 但是需要你连接那些正在运行SMTP协议的服务器，在服务器或本地机器上设置SMTP客户端有点儿复杂，也超出了本书的介绍范围，但是有很多资料可以帮助解决问题。

下面的代码运行的前提是你的电脑已经可以正常地运行一个SMTP客户端**（如果要调整代码用于远程SMTP客户端，请把localhost改成远程服务器地址）**

## 用python发一封邮件

以下内容参考[廖雪峰教程](https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001432005226355aadb8d4b2f3f42f6b1d6f2c5bd8d5263000)

python对SMTP的支持有smtplib和email两个模块， email负责构造邮件，smtplib负责发送邮件。

### 构造一个简单的纯文本邮件 

In [ ]:
from email.mime.text import MIMEText
from email import  encoders
from email.header import  Header
from email.utils import parseaddr, formataddr




- 第一个参数是邮件正文，第二个参数是MIME的subtype，传入'plain'表示纯文本，最后用utf-8编码保证多语言兼容性

### 通过SMTP发送

In [ ]:
def _format_addr(s):
    name, addr = parseaddr(s)
    return formataddr((Header(name, 'utf-8').encode(), addr))
msg = MIMEText('Life is short, I use python', 'plain', 'utf-8')
# msg['From'] = _format_addr("jdshen")
msg['To'] = _format_addr("2412697841@qq.com")
msg['Subject'] = Header("Hello!", 'utf-8').encode()

# 输入Email地址和口令
from_addr = "m18896921739@163.com"
passwd = input("Passwd: ")
# 输入收件人地址
to_addr = "2412697841@qq.com"
# 输入smtp服务器地址
smtp_server = "smtp.163.com"

import smtplib
server = smtplib.SMTP(smtp_server, 25)  # smtp协议的默认端口是25
server.set_debuglevel(1)
server.login(from_addr, passwd)
server.sendmail(from_addr, [to_addr], msg.as_string())
server.quit()

- 使用set_debuglevel(1)就可以打印出和SMTP服务器交互的所有信息。SMTP协议就是简单的文本命令和响应，login()方法用来登录服务器，sendmail()方法是发邮件，一次可以发给多个人，所以传入一个list， 邮件正文是一个str，as_string()把MIMEText对象变成str

** <span class="mark">特别注意：邮件的to和subject一定要加上去，不然会出现545错误，还有发给其他邮箱很容易被当成垃圾邮件，需要在垃圾箱里设置为信任，以后发给这个邮箱的邮件就不会进垃圾箱了。</span>**

### 发送附件

带附件的邮件可以看成包含若干部分的邮件：文本和各个附件本身
所以可以构造一个MIMEMultipart对象代表邮件本身，然后往里面加上一个MIMEText作为邮件正文，再继续往里面加上表示附件的MIMIBase对象即可

In [ ]:
from email.mime.multipart import MIMEMultipart, MIMEBase
def _format_addr(s):
    name, addr = parseaddr(s)
    return formataddr((Header(name, 'utf-8').encode(), addr))

msg = MIMEMultipart()
# msg['From'] = _format_addr("jdshen")
msg['To'] = _format_addr("2412697841@qq.com")
msg['Subject'] = Header("Hello!", 'utf-8').encode()

# 添加邮件正文
msg.attach(MIMEText('Life is short, I use python', 'plain', 'utf-8'))

# 添加图片：
# 从本地读取一个图片
with open("book.jpg", 'rb') as f:
    mime = MIMEBase('image', 'png', filename='test.png')
    # 加上必要的头信息
    mime.add_header('Content-Disposition', 'attachment', filename='test.png')
    mime.add_header('Content-ID', '<0>')
    mime.add_header('X-Attachment-Id', '0')
    # 读取附件的内容
    mime.set_payload(f.read())
    # 用Base64编码
    encoders.encode_base64(mime)
    msg.attach(mime)

# 输入Email地址和口令
from_addr = "m18896921739@163.com"
passwd = input("Passwd: ")
# 输入收件人地址
to_addr = "2412697841@qq.com"
# 输入smtp服务器地址
smtp_server = "smtp.163.com"

import smtplib
server = smtplib.SMTP(smtp_server, 25)  # smtp协议的默认端口是25
server.set_debuglevel(1)
server.login(from_addr, passwd)
server.send_message(msg, )from_addr, [to_addr]
server.sendmail(from_addr, [to_addr], msg)
server.quit()

### 可使用格式为html的邮件文本

In [ ]:
from email.mime.multipart import MIMEMultipart, MIMEBase
def _format_addr(s):
    name, addr = parseaddr(s)
    return formataddr((Header(name, 'utf-8').encode(), addr))

# 添加邮件正文
msg = MIMEMultipart('alternative')
msg.attach(MIMEText('Life is short, I use python', 'plain', 'utf-8'))
msg.attach(MIMEText('<html><body><h1>Hello</h1>' +
    '<p><img src="cid:0"></p>' +
    '</body></html>', 'html', 'utf-8'))



# msg['From'] = _format_addr("jdshen")
msg['To'] = _format_addr("2412697841@qq.com")
msg['Subject'] = Header("Hello!", 'utf-8').encode()


# 输入Email地址和口令
from_addr = "m18896921739@163.com"
passwd = input("Passwd: ")
# 输入收件人地址
to_addr = "2412697841@qq.com"
# 输入smtp服务器地址
smtp_server = "smtp.163.com"

import smtplib
server = smtplib.SMTP(smtp_server, 25)  # smtp协议的默认端口是25
server.set_debuglevel(1)
server.login(from_addr, passwd)
server.sendmail(from_addr, [to_addr], msg.as_string())
server.quit()

## 使用POP3收取邮件